In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
conn=sqlite3.connect('sample_rental.db')
cursor=conn.cursor()

* buscar actores con mismo nombre y apellido y diferente actor_id

In [3]:
q=(""" 
 SELECT actor_id, first_name, last_name
 FROM actor
 WHERE (first_name, last_name) 
    in(
    SELECT first_name, last_name
    FROM actor
    GROUP BY first_name, last_name
    HAVING COUNT (actor_id)>1
    )
"""
)
df=pd.read_sql(q,conn)
df

,actor_id,first_name,last_name
0,101,SUSAN,DAVIS
1,110,SUSAN,DAVIS


* contar actores por cada pelicula

In [4]:
q=""" 
    WITH t1 AS(
    SELECT title, first_name, last_name ||""|| last_name as full_name
    FROM film
    INNER JOIN film_actor
    ON film.film_id = film_actor.film_id
    INNER JOIN actor
    ON actor.actor_id = film_actor.actor_id
    GROUP BY title, first_name, last_name
    ORDER By title
    )
    SELECT 
        title, 
        count(full_name) as count_of_actors
    FROM t1
    GROUP BY 1  
    ORDER BY 2 DESC
    LIMIT 10
"""

df=pd.read_sql(q,conn)
df

,title,count_of_actors
0,LAMBS CINCINATTI,15
1,RANDOM GO,13
2,MUMMY CREATURES,13
3,DRACULA CRYSTAL,13
4,CRAZY HOME,13
5,CHITTY LOCK,13
6,BOONDOCK BALLROOM,13
7,TITANIC BOONDOCK,12
8,SKY MIRACLE,12
9,LONELY ELEPHANT,12


* agrupar por full name y agrupar por número de películas

In [6]:
q = """
WITH t1 AS (
      SELECT first_name || " " || last_name as full_name,
        count(film.film_id) as movie_count,
        dense_rank() OVER (ORDER BY COUNT(film.film_id) desc) as d_rank
        FROM film
        INNER JOIN film_actor 
        ON film.film_id = film_actor.film_id
        INNER JOIN actor
        ON actor.actor_id = film_actor.actor_id
        GROUP BY full_name
        ORDER BY 2 DESC
      )
  
  
  
  SELECT 
    full_name as nombre
    ,movie_count as conteo
    ,d_rank as rank 
  FROM t1
  WHERE d_rank <= 10;
    """
df = pd.read_sql(q, conn)
df

,nombre,conteo,rank
0,SUSAN DAVIS,54,1
1,GINA DEGENERES,42,2
2,WALTER TORN,41,3
3,MARY KEITEL,40,4
4,MATTHEW CARREY,39,5
5,SANDRA KILMER,37,6
6,SCARLETT DAMON,36,7
7,VIVIEN BASINGER,35,8
8,VAL BOLGER,35,8
9,UMA WOOD,35,8


In [7]:
df.columns

Index(['nombre', 'conteo', 'rank'], dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  30 non-null     object
 1   conteo  30 non-null     int64 
 2   rank    30 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 848.0+ bytes


In [9]:
df_new=df.loc[df['rank']>5]
#copia en df_new el df original con rank>5

* veces que una pelicula ha pasado de disponible a alquiler y mostrar el precio

#### REVISAR

In [10]:
q=""" 
SELECT * FROM RENTAL
WHERE return_date is Null OR (JULIANDAY(return_date)-JULIANDAY(rental_date)>7)
"""
df = pd.read_sql(q, conn)
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,3,2005-05-24 23:03:39.000,1711,408,2005-06-01 22:12:39.000,1,2023-05-27 11:47:04
1,4,2005-05-24 23:04:41.000,2452,333,2005-06-03 01:43:41.000,2,2023-05-27 11:47:04
2,5,2005-05-24 23:05:21.000,2079,222,2005-06-02 04:33:21.000,1,2023-05-27 11:47:04
3,11,2005-05-25 00:09:02.000,4443,142,2005-06-02 20:56:02.000,2,2023-05-27 11:47:04
4,15,2005-05-25 00:39:22.000,3049,319,2005-06-03 03:30:22.000,1,2023-05-27 11:47:04
...,...,...,...,...,...,...,...
4672,16037,2005-08-23 22:13:04.000,341,45,2005-09-01 02:48:04.000,2,2023-05-27 11:49:43
4673,16039,2005-08-23 22:18:51.000,545,78,2005-08-31 19:55:51.000,2,2023-05-27 11:49:43
4674,16040,2005-08-23 22:19:33.000,3524,195,2005-09-02 02:19:33.000,2,2023-05-27 11:49:43
4675,16043,2005-08-23 22:21:03.000,3869,526,2005-08-31 03:09:03.000,2,2023-05-27 11:49:43


* clientes que no han devuelto un dvd

In [11]:
q=""" 
SELECT 
    c.first_name
    ,c.last_name
    ,c.email
    ,c.active
    ,COUNT (r.rental_id) as no_devueltos
FROM customer as c
INNER JOIN rental as r
ON c.customer_id=r.customer_id
WHERE r.return_date IS NULL and c.active=0
GROUP BY R.customer_id
HAVING no_devueltos >= 1
ORDER BY 3
"""
df = pd.read_sql(q, conn)
df

,first_name,last_name,email,active,no_devueltos
0,CHRISTIAN,JUNG,CHRISTIAN.JUNG@sakilacustomer.org,0,1
1,JUDITH,COX,JUDITH.COX@sakilacustomer.org,0,1
2,KENNETH,GOODEN,KENNETH.GOODEN@sakilacustomer.org,0,1
3,TERRANCE,ROUSH,TERRANCE.ROUSH@sakilacustomer.org,0,1


#### 1.¿Tenemos actores en la tabla de actores que comparten el nombre completo y, en caso afirmativo, muestran esos nombres compartidos?

In [12]:
q=(""" 
 SELECT actor_id, first_name, last_name
 FROM actor
 WHERE (first_name, last_name) 
    in(
    SELECT first_name, last_name
    FROM actor
    GROUP BY first_name, last_name
    HAVING COUNT (actor_id)>1
    )
"""
)
df=pd.read_sql(q,conn)
df

,actor_id,first_name,last_name
0,101,SUSAN,DAVIS
1,110,SUSAN,DAVIS


In [14]:
## otra forma
q=""" 
SELECT
DISTINCT a1.first_name, a1.last_name
FROM actor as a1 JOIN actor as a2
WHERE (a1.first_name=a2.first_name) AND (a1.last_name=a2.last_name) AND (a1.actor_id!=a2.actor_id)
"""
df = pd.read_sql(q, conn)
df

,first_name,last_name
0,SUSAN,DAVIS


In [15]:
q=""" 
SELECT
    COUNT(DISTINCT first_name||" "|| last_name) as Nombres_distintos
FROM actor;
"""
df = pd.read_sql(q, conn)
df

,Nombres_distintos
0,199


#### 2.Muestra los nombres de los clientes que comparten la misma dirección (por ejemplo, esposo y esposa)
Para responder a esta pregunta, debemos buscar más de un cliente que tenga el mismo address_id pero diferentes customer_ids. Eso significa que haremos JOIN a la tabla de clientes para sí mismo (autounión).

In [16]:
q="""
select 
count(address_id) from address
group by address_id
order by address_id
"""

df = pd.read_sql(q, conn)
df

,count(address_id)
0,1
1,1
2,1
3,1
4,1
...,...
598,1
599,1
600,1
601,1


In [17]:
q="""
select 
address_id, count(address_id) from customer
group by address_id
order by address_id
"""

df = pd.read_sql(q, conn)
df

,address_id,count(address_id)
0,5,1
1,6,1
2,7,1
3,8,1
4,9,1
...,...,...
594,601,1
595,602,1
596,603,1
597,604,1


#### 3.Muestra el monto total pagado por todos los clientes en la tabla de pagos.
Utilizar aquí la función agregada SUM() en la columna de monto de la tabla de pagos

In [18]:
q=""" 
SELECT SUM(amount) as Total_pagado
FROM payment
"""
df = pd.read_sql(q, conn)
df

,Total_pagado
0,67416.51


##### 4. Muestre el monto total pagado por cada cliente en la tabla de pagos.

In [20]:
q=""" 
SELECT c.first_name||" "||c.last_name as Nombre, SUM(p.amount) as Total_pagado
FROM payment as p
INNER JOIN customer as c
WHERE c.customer_id=p.customer_id
GROUP BY c.customer_id
ORDER BY Nombre ASC
"""
df = pd.read_sql(q, conn)
df

,Nombre,Total_pagado
0,AARON SELBY,110.76
1,ADAM GOOCH,101.78
2,ADRIAN CLARY,74.81
3,AGNES BISHOP,98.77
4,ALAN KAHN,124.74
...,...,...
594,WILLIE MARKHAM,101.75
595,WILMA RICHARDS,91.80
596,YOLANDA WEAVER,110.73
597,YVONNE WATKINS,92.79


##### 5. Pago total mas alto

In [21]:
q=""" 
SELECT c.first_name||" "||c.last_name as Nombre, SUM(p.amount) as Total_pagado
FROM payment as p
INNER JOIN customer as c
WHERE c.customer_id=p.customer_id
GROUP BY c.customer_id
ORDER BY SUM(p.amount) DESC
LIMIT 1
"""
df = pd.read_sql(q, conn)
df

,Nombre,Total_pagado
0,KARL SEAL,221.55


##### 6. 